In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import seaborn as sns

In [3]:
df  = pd.read_csv('/content/netflix.csv')

In [4]:
df.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."


## 1. Content Strategy

Question 1: What is the ratio of Movies vs TV Shows on Netflix?


In [5]:
type_count = df['type'].value_counts()
total_content = type_count.sum()
type_percentage = (type_count / total_content) * 100
print(f'Total Contents: {total_content}\n')
print(f'Type Count:\n{type_count}\n')
print(f'Type Percentage:\n{type_percentage}\n')

Total Contents: 8807

Type Count:
type
Movie      6131
TV Show    2676
Name: count, dtype: int64

Type Percentage:
type
Movie      69.615079
TV Show    30.384921
Name: count, dtype: float64



2. Which genres are most popular on Netflix globally?

In [7]:
genres = df['listed_in'].str.split(',').explode().str.strip()
popular_genres = genres.value_counts()
print("Top 10 Most Common Genres:")
print(popular_genres.head(10))

Top 10 Most Common Genres:
listed_in
International Movies        2752
Dramas                      2427
Comedies                    1674
International TV Shows      1351
Documentaries                869
Action & Adventure           859
TV Dramas                    763
Independent Movies           756
Children & Family Movies     641
Romantic Movies              616
Name: count, dtype: int64


3: Years with Highest Content Release

In [8]:
release_year_counts = df['release_year'].value_counts()
print("Top 10 Release Years for Content on Netflix:")
print(release_year_counts.head(10))


Top 10 Release Years for Content on Netflix:
release_year
2018    1147
2017    1032
2019    1030
2020     953
2016     902
2021     592
2015     560
2014     352
2013     288
2012     237
Name: count, dtype: int64


4: Countries Producing the Most Content

In [9]:
# Drop rows where 'country' is missing
countries_df = df.dropna(subset=['country'])
# Split, explode, and count individual countries
countries = countries_df['country'].str.split(',').explode().str.strip()
popular_countries = countries.value_counts()
print("Top 10 Content Producing Countries:")
# Filter out empty strings if any
popular_countries = popular_countries[popular_countries.index != '']
print(popular_countries.head(10))

Top 10 Content Producing Countries:
country
United States     3690
India             1046
United Kingdom     806
Canada             445
France             393
Japan              318
Spain              232
South Korea        231
Germany            226
Mexico             169
Name: count, dtype: int64


5: Trend of Adding New Content Year by Year

In [15]:
added_df = df.dropna(subset=['date_added']).copy()
added_df['date_added'] = pd.to_datetime(added_df['date_added'], errors='coerce')
added_df = added_df.dropna(subset=['date_added'])
added_df['year_added'] = added_df['date_added'].dt.year
added_df['year_added'] = added_df['year_added'].astype(int)
content_added_trend = added_df['year_added'].value_counts().sort_index(ascending=False)
print("Content Added to Netflix by Year:")
print(content_added_trend)

Content Added to Netflix by Year:
year_added
2021    1498
2020    1878
2019    1999
2018    1625
2017    1164
2016     418
2015      73
2014      23
2013      10
2012       3
2011      13
2010       1
2009       2
2008       2
Name: count, dtype: int64


## 2. User Demographics & Targeting

In [16]:
# --- Q6: Which ratings are most frequent? ---
print("\nQ6: Which ratings (e.g., TV-MA, PG, etc.) are most frequent?")
rating_counts = df.dropna(subset=['rating'])['rating'].value_counts()
print("Top 10 Most Frequent Ratings:")
print(rating_counts.head(10))
print("-" * 50)

# --- Q7: Do some countries tend to produce more mature content (TV-MA)? ---
print("\nQ7: Which countries produce the most TV-MA content?")
mature_df = df[df['rating'] == 'TV-MA'].dropna(subset=['country'])
mature_countries = mature_df['country'].str.split(',').explode().str.strip()
mature_country_counts = mature_countries.value_counts()
print("Top 10 Countries Producing TV-MA Content:")
print(mature_country_counts.head(10))
print("-" * 50)

# --- Q8: Which genres are more associated with TV Shows vs Movies? ---
print("\nQ8: Which genres are more associated with TV Shows vs Movies?")
movies_df_genres = df[df['type'] == 'Movie'].dropna(subset=['listed_in'])
movie_genres = movies_df_genres['listed_in'].str.split(',').explode().str.strip().value_counts()

shows_df_genres = df[df['type'] == 'TV Show'].dropna(subset=['listed_in'])
show_genres = shows_df_genres['listed_in'].str.split(',').explode().str.strip().value_counts()

print("Top 5 Genres for Movies:")
print(movie_genres.head(5))
print("\nTop 5 Genres for TV Shows:")
print(show_genres.head(5))
print("-" * 50)

# --- Q9: Which genres dominate the U.S. vs other countries? ---
print("\nQ9: Which genres dominate the U.S. vs non-U.S. countries?")
us_genres = df[df['country'].str.contains('United States', na=False)]['listed_in'].str.split(',').explode().str.strip().value_counts()

# Select rows that DO NOT contain 'United States' AND have a non-null country
non_us_df = df[~df['country'].str.contains('United States', na=True) & df['country'].notna()]
non_us_genres = non_us_df['listed_in'].str.split(',').explode().str.strip().value_counts()

print("Top 5 Genres in the United States:")
print(us_genres.head(5))
print("\nTop 5 Genres outside the United States (in content with specified country):")
print(non_us_genres.head(5))
print("-" * 50)

# --- Q10: What genres are most popular in the last 3 years? ---
print("\nQ10: What genres are most popular in the last 3 release years?")
# Get the 3 most recent *release years* present in the data
max_year = df['release_year'].max()
recent_years = [max_year, max_year - 1, max_year - 2]
print(f"(Based on release years {recent_years})")

recent_df = df[df['release_year'].isin(recent_years)]
recent_genres = recent_df['listed_in'].str.split(',').explode().str.strip().value_counts()
print("Top 10 Genres from the Last 3 Release Years:")
print(recent_genres.head(10))
print("-" * 50)


Q6: Which ratings (e.g., TV-MA, PG, etc.) are most frequent?
Top 10 Most Frequent Ratings:
rating
TV-MA    3207
TV-14    2160
TV-PG     863
R         799
PG-13     490
TV-Y7     334
TV-Y      307
PG        287
TV-G      220
NR         80
Name: count, dtype: int64
--------------------------------------------------

Q7: Which countries produce the most TV-MA content?
Top 10 Countries Producing TV-MA Content:
country
United States     1101
India              266
United Kingdom     253
Spain              170
France             163
Canada             107
Mexico             102
Japan              101
South Korea         92
Germany             79
Name: count, dtype: int64
--------------------------------------------------

Q8: Which genres are more associated with TV Shows vs Movies?
Top 5 Genres for Movies:
listed_in
International Movies    2752
Dramas                  2427
Comedies                1674
Documentaries            869
Action & Adventure       859
Name: count, dtype: int64

Top 

3. Talent Acquisition & Partnerships

In [19]:
# --- Q11: Who are the top 10 directors with the most Netflix content? ---
print("\nQ11: Who are the top 10 directors with the most Netflix content?")
directors_df = df.dropna(subset=['director'])
directors = directors_df['director'].str.split(',').explode().str.strip()
# Filter out 'Unknown' or other placeholders if they exist as strings
directors = directors[directors != 'Unknown']
top_directors = directors.value_counts()
print("Top 10 Directors:")
print(top_directors.head(10))
print("-" * 50)

# --- Q12: Which actors appear most frequently in Netflix shows? ---
print("\nQ12: Which actors appear most frequently in Netflix content?")
cast_df = df.dropna(subset=['cast'])
actors = cast_df['cast'].str.split(',').explode().str.strip()
actors = actors[actors != 'Unknown'] # Filter out placeholders
top_actors = actors.value_counts()
print("Top 10 Actors:")
print(top_actors.head(10))
print("-" * 50)

# --- Q13: Which director-genre pairs are most frequent? ---
print("\nQ13: Which director-genre pairs are most frequent?")
# Start with a clean DataFrame, dropping rows missing either value
talent_df = df.dropna(subset=['director', 'listed_in']).copy()

# --- FIX for ValueError ---
# Split the strings into lists first
talent_df['director_list'] = talent_df['director'].str.split(',')
talent_df['genre_list'] = talent_df['listed_in'].str.split(',')

# Explode the DataFrame first on directors, then on genres
# This creates rows for every director-show/movie combination
talent_df = talent_df.explode('director_list')
# This further explodes, creating rows for every director-genre combination
talent_df = talent_df.explode('genre_list')

# Now, strip whitespace from the exploded columns
talent_df['director_single'] = talent_df['director_list'].str.strip()
talent_df['genre_single'] = talent_df['genre_list'].str.strip()
# --- END FIX ---

# Filter out placeholders and potential NaNs from stripping
talent_df = talent_df[talent_df['director_single'] != 'Unknown']
talent_df = talent_df.dropna(subset=['director_single', 'genre_single'])

director_genre_counts = talent_df.groupby(['director_single', 'genre_single']).size().sort_values(ascending=False)
print("Top 10 Director-Genre Pairs:")
print(director_genre_counts.head(10))
print("-" * 50)

# --- Q14: How many titles have unknown directors or cast members? ---
print("\nQ14: How many titles have unknown directors or cast members?")
unknown_directors = df['director'].isnull().sum()
unknown_cast = df['cast'].isnull().sum()
print(f"Titles with unknown (NaN) directors: {unknown_directors}")
print(f"Titles with unknown (NaN) cast: {unknown_cast}")
print("-" * 50)


Q11: Who are the top 10 directors with the most Netflix content?
Top 10 Directors:
director
Rajiv Chilaka          22
Jan Suter              21
Raúl Campos            19
Suhas Kadav            16
Marcus Raboy           16
Jay Karas              15
Cathy Garcia-Molina    13
Martin Scorsese        12
Youssef Chahine        12
Jay Chapman            12
Name: count, dtype: int64
--------------------------------------------------

Q12: Which actors appear most frequently in Netflix content?
Top 10 Actors:
cast
Anupam Kher         43
Shah Rukh Khan      35
Julie Tejwani       33
Takahiro Sakurai    32
Naseeruddin Shah    32
Rupa Bhimani        31
Om Puri             30
Akshay Kumar        30
Yuki Kaji           29
Amitabh Bachchan    28
Name: count, dtype: int64
--------------------------------------------------

Q13: Which director-genre pairs are most frequent?
Top 10 Director-Genre Pairs:
director_single      genre_single            
Rajiv Chilaka        Children & Family Movies    22
Ja

In [20]:
# --- Q15: What is the average duration of Movies on Netflix? ---
print("\nQ15: What is the average duration of Movies on Netflix?")
movies_df_dur = df[df['type'] == 'Movie'].copy()
# Clean 'duration' column (e.g., "90 min") and convert to numeric
movies_df_dur['duration_min'] = pd.to_numeric(movies_df_dur['duration'].str.replace(' min', ''), errors='coerce')
movies_df_dur = movies_df_dur.dropna(subset=['duration_min']) # Drop rows that couldn't be converted
avg_duration = movies_df_dur['duration_min'].mean()
print(f"Average movie duration: {avg_duration:.2f} minutes")
print("-" * 50)

# --- Q16: What’s the most common number of seasons for TV shows? ---
print("\nQ16: What’s the most common number of seasons for TV shows?")
shows_df_dur = df[df['type'] == 'TV Show'].dropna(subset=['duration'])
season_counts = shows_df_dur['duration'].value_counts()
print("Most Common Season Counts:")
print(season_counts.head(5))
if not season_counts.empty:
  print(f"The most common is: {season_counts.idxmax()}")
else:
  print("No TV Show duration data found.")
print("-" * 50)

# --- Q17: Is there a trend in movie durations over the years? ---
print("\nQ17: Is there a trend in movie durations over the years? (by release year)")
# Use movies_df_dur from Q15
duration_trend = movies_df_dur.dropna(subset=['release_year', 'duration_min']).groupby('release_year')['duration_min'].mean()
print("Average Movie Duration by Release Year (Top 20 recent):")
print(duration_trend.sort_index(ascending=False).head(20).round(2))
print("-" * 50)


Q15: What is the average duration of Movies on Netflix?
Average movie duration: 99.58 minutes
--------------------------------------------------

Q16: What’s the most common number of seasons for TV shows?
Most Common Season Counts:
duration
1 Season     1793
2 Seasons     425
3 Seasons     199
4 Seasons      95
5 Seasons      65
Name: count, dtype: int64
The most common is: 1 Season
--------------------------------------------------

Q17: Is there a trend in movie durations over the years? (by release year)
Average Movie Duration by Release Year (Top 20 recent):
release_year
2021     96.44
2020     92.14
2019     93.47
2018     96.19
2017     95.54
2016     95.36
2015     99.51
2014    100.26
2013     98.05
2012    100.76
2011    102.87
2010    104.46
2009    108.95
2008    106.24
2007    113.23
2006    113.01
2005    113.93
2004    114.33
2003    116.27
2002    119.55
Name: duration_min, dtype: float64
--------------------------------------------------


##5. Content Launch Strategy

In [23]:
# --- Q18: In which months does Netflix add the most content? ---
print("\nQ18: In which months does Netflix add the most content?")
# Use 'added_df' from Q5 which is already cleaned
added_df['month_added'] = added_df['date_added'].dt.month_name()
month_counts = added_df['month_added'].value_counts()
# Order by month for better readability
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
print(month_counts.reindex(month_order))
print("-" * 50)

# --- Q19: How does the genre distribution vary across different years? ---
# Start with a clean DataFrame, dropping rows missing either value
df_genre_year = df.dropna(subset=['release_year', 'listed_in']).copy()

# Split 'listed_in' into a list
df_genre_year['genre_list'] = df_genre_year['listed_in'].str.split(',')

# Explode the DataFrame on the new list column
df_genre_year = df_genre_year.explode('genre_list')

# Now create the final 'genre' column by stripping whitespace
df_genre_year['genre'] = df_genre_year['genre_list'].str.strip()


# Drop any potential NaNs created by stripping or empty strings
df_genre_year = df_genre_year.dropna(subset=['genre'])
df_genre_year = df_genre_year[df_genre_year['genre'] != '']

# Get 5 most common release years in the dataset
common_years = df_genre_year['release_year'].value_counts().head(5).index

for year in common_years:
  print(f"\nTop 5 genres for release year {int(year)}:")
  year_genres = df_genre_year[df_genre_year['release_year'] == year]['genre'].value_counts().head(5)
  print(year_genres)
print("-" * 50)

# --- Q20: Which countries produce the most content in each genre? ---
print("\nQ20: Which countries produce the most content in each (top 5) genre?")
# Start with a clean DataFrame, dropping rows missing either value
df_country_genre = df.dropna(subset=['country', 'listed_in']).copy()

# Split the strings into lists first
df_country_genre['country_list'] = df_country_genre['country'].str.split(',')
df_country_genre['genre_list'] = df_country_genre['listed_in'].str.split(',')

# Explode the DataFrame first on countries, then on genres
df_country_genre = df_country_genre.explode('country_list')
df_country_genre = df_country_genre.explode('genre_list')

# Now, strip whitespace from the exploded columns
df_country_genre['country_single'] = df_country_genre['country_list'].str.strip()
df_country_genre['genre_single'] = df_country_genre['genre_list'].str.strip()

# Filter out empty country strings and genres
df_country_genre = df_country_genre[df_country_genre['country_single'] != '']
df_country_genre = df_country_genre[df_country_genre['genre_single'] != '']
df_country_genre = df_country_genre.dropna(subset=['country_single', 'genre_single'])

# Use popular_genres from Q2
top_genres_list = popular_genres.head(5).index.tolist()

for genre in top_genres_list:
  print(f"\nTop 3 countries producing '{genre}':")
  top_countries = df_country_genre[df_country_genre['genre_single'] == genre]['country_single'].value_counts().head(3)
  print(top_countries)
print("-" * 50)








Q18: In which months does Netflix add the most content?
month_added
January      727
February     557
March        734
April        759
May          626
June         724
July         819
August       749
September    765
October      755
November     697
December     797
Name: count, dtype: int64
--------------------------------------------------

Top 5 genres for release year 2018:
genre
International Movies      340
Dramas                    304
International TV Shows    190
Comedies                  178
Independent Movies        131
Name: count, dtype: int64

Top 5 genres for release year 2019:
genre
International Movies      282
Dramas                    243
International TV Shows    201
Comedies                  159
TV Dramas                 133
Name: count, dtype: int64

Top 5 genres for release year 2017:
genre
International Movies      328
Dramas                    285
Documentaries             172
Comedies                  164
International TV Shows    136
Name: count, dtype: